In [0]:
import pandas  as pd
import numpy  as  np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

# Load data

In [19]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

# Remove prices in EUR

In [22]:
[x for x in df.columns if 'price' in x]

['price_currency', 'price_details', 'price_value']

In [23]:
df.price_currency.value_counts()

PLN    106290
EUR       204
Name: price_currency, dtype: int64

In [24]:
df = df[ df['price_currency'] != 'EUR' ]
df.shape

(106290, 155)

# Dummy model

In [21]:
feats = ['car_id']

X = df[feats].values # training data
y = df['price_value'].values # target values

model = DummyRegressor()
model.fit(X , y)
y_pred = model.predict(X)

# benchmark value for future model
mae(y_true = y, y_pred=y_pred)

39465.934630440985

# Features

In [25]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[ feat ][0], list): continue

  factorized_values = df[ feat ].factorize()[0]
  # avoid columns named like 'feat___cat___cat'
  if SUFFIX_CAT in feat:
    df[ feat ] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [30]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

# First model

In [31]:
X = df[ cat_feats ].values
y = df[ 'price_value' ].values

model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X, y, cv = 5, scoring='neg_mean_absolute_error')
np.mean(scores)

-19587.598553433912

# Feature importance

In [33]:
m = DecisionTreeRegressor(max_depth=5)
m.fit(X, y)

imp = PermutationImportance(m, random_state=0).fit(X, y)
eli5.show_weights(imp, feature_names=cat_feats)

Weight,Feature
0.2533 ± 0.0040,param_napęd__cat
0.2008 ± 0.0047,param_faktura-vat__cat
0.1943 ± 0.0088,param_stan__cat
0.1423 ± 0.0086,param_rok-produkcji__cat
0.0629 ± 0.0047,param_moc__cat
0.0424 ± 0.0014,feature_kamera-cofania__cat
0.0412 ± 0.0008,param_skrzynia-biegów__cat
0.0286 ± 0.0037,param_marka-pojazdu__cat
0.0191 ± 0.0022,param_pojemność-skokowa__cat
0.0163 ± 0.0005,feature_bluetooth__cat


In [40]:
!git push origin master

Counting objects: 4, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 260.02 KiB | 7.22 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/michkam89/dw-matrix.git
   8a10bde..2f7c38b  master -> master
